In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install selenium
!apt update
!apt install chromium-chromedriver

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,473 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,278 kB]
Fetched 2,979 kB in 2s (1,479 kB/s)
Reading package lists... Done
Buildin

In [5]:
import requests
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
import lxml.html as lxl
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [27]:
MAX_PAGE = 1

# 1. Functions

## Scraping the webpages to get all the car urls

In [28]:
## Get all the urls for all the listed used vehicles on truecar.com
def urls_scraping(base_url = 'https://www.truecar.com/used-cars-for-sale/listings/'):
    urls = []
    pages = []
    for i in range(1, MAX_PAGE+1):
        pages.append(base_url + '?page=' + str(i))
    for page in pages:
        try:
            response = requests.get(page)
            response.raise_for_status()
        except:
            break
        root = lxl.fromstring(response.content)
        url = ['https://www.truecar.com' + link for link in root.xpath('//div[@data-test="cardContent"]/a/@href')]
        urls += url

    return urls

## Config Web Drive to Crawl

In [7]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


## Parse one url to get information and return a dataframe

In [24]:
def page_scraping(urls):
  dr = webdriver.Chrome(options=options)
  data = []
  for url in urls:
    dr.get(url)
    title = dr.find_elements(By.CLASS_NAME, "heading-md-2")[0]
    year = title.text.split(" ")[0]
    make = title.text.split(" ")[1]
    model = title.text.split(" ")[2]
    sub_model = ' '.join(title.text.split(" ")[3:])
    location = dr.find_elements(By.CSS_SELECTOR, ".mr-1")[0].text
    city = location.split(", ")[0]
    state = location.split(", ")[1]
    mileage = dr.find_elements(By.XPATH, '//*[@id="mainContent"]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div[3]/div/div')[0].text.split(" ")[0]
    price = dr.find_elements(By.XPATH, '//div[contains(@class, "heading-2") and contains(@data-test, "vdpPricingInfoPrice")]')[0].text.replace("$", "")
    exterior_color = dr.find_elements(By.XPATH, '//*[@id="mainContent"]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div[1]/div/div[contains(@class, "items-center")]')[0].text.replace("Exterior: ", "")
    interior_color = dr.find_elements(By.XPATH, '//*[@id="mainContent"]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div[2]/div/div[contains(@class, "items-center")]')[0].text.replace("Interior: ", "")
    mpg = dr.find_elements(By.XPATH, '//*[@id="mainContent"]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div[5]/div/div')[0].text.split(" ")
    mpg_city = mpg[0]
    mpg_hwy = mpg[3]
    engine = dr.find_elements(By.XPATH, '//*[@id="mainContent"]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div[8]/div/div')[0].text
    transmission = dr.find_elements(By.XPATH, '//*[@id="mainContent"]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div[6]/div/div')[0].text
    drive_type = dr.find_elements(By.XPATH, '//*[@id="mainContent"]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div[7]/div/div')[0].text
    fuel_type = dr.find_elements(By.XPATH, '//*[@id="mainContent"]/div[2]/div[2]/div[2]/div[2]/div[2]/div/div[4]/div/div')[0].text.replace("Fuel Type: ", "")
    button = dr.find_element(By.XPATH, "//button[@class='link-button mb-4']")
    dr.execute_script("arguments[0].click();", button)

    wait = WebDriverWait(dr, 5)
    popup_elements = wait.until(EC.visibility_of_all_elements_located((By.XPATH, "//div[@class='modal-body modal-body-lg']")))[0]
    features = popup_elements.find_elements(By.XPATH, "//div[@class='heading-3 normal-case text-base']")
    popular_feature = ''
    for i in features:
      popular_feature = popular_feature + ", " + i.text
    vehicle_histories = popup_elements.find_elements(By.XPATH, "//div[@class='grow space-y-1']/div[@class='text-sm']")[0].text.split('·')
    accidents = vehicle_histories[0].split(' ')[0]
    owners = vehicle_histories[1].split(' ')[0]
    print("Year="+year+", Make="+make+", Model="+model+", Sub model:"+ sub_model+", City="+ city+ ", State="+state+", Mileage="+ mileage+", Price="+ price+ "\
    , Exterior_color="+exterior_color+"\n, interior_color="+ interior_color+", mpg_city="+mpg_city, "mpg_hwy="+mpg_hwy+", Engine="+engine+", \
    Transmission="+ transmission + ", Drive_type="+ drive_type + ", Fuel_type="+ fuel_type + ",\n Popular_feature="+ popular_feature + ", accidents="+ accidents + ", Owners="+ owners)

    data.append({'year':year, 'make': make, 'model': model, 'sub_model': sub_model, 'city':city, 'state': state,
          'mileage': mileage, 'price': price, 'exterior_color': exterior_color,
                    'interior_color': interior_color, 'mpg_city': mpg_city, 'mpg_hwy': mpg_hwy, 'engine': engine,
                    'transmission': transmission, 'drive_type': drive_type, 'fuel_type': fuel_type,
                    'popular_feature': popular_feature, 'accidents': accidents, 'owners': owners})
  dr.quit()
  return pd.DataFrame(data)




## Use multi-processing to scrape all urls and merge into one dataframe

In [19]:
# Use multi-processing to speed up the web-scraping
# fully make use of 8 cores of my macbook pro.
from multiprocessing import Pool
num_partitions = 32
num_cores = 8
def parallelize(urls, func):
    url_set = np.array_split(urls, num_partitions)
    pool = Pool(num_cores)
    df = pd.concat(pool.map(func, url_set))
    pool.close()
    pool.join()
    return df
# main function to scrape all the urls and merge all the data into one dataframe
def scraping(urls):
    scraping_data = [page_scraping(url) for url in urls]
    return pd.concat(scraping_data, axis=1).T

# 2. Main function

## Getting all urls

In [29]:
%%time
urls=urls_scraping() # extract all vehicle urls from allowed 10 pages.
urls[:100]

CPU times: user 145 ms, sys: 7.67 ms, total: 152 ms
Wall time: 1.39 s


['https://www.truecar.com/used-cars-for-sale/listing/55SWF4JB6HU216138/2017-mercedes-benz-c-class/?sponsoredVehiclePosition=0&zipcode=92504',
 'https://www.truecar.com/used-cars-for-sale/listing/1FTEW1E55LFA86616/2020-ford-f-150/?sponsoredVehiclePosition=1&zipcode=49254',
 'https://www.truecar.com/used-cars-for-sale/listing/1FTEW1EP6GFA60291/2016-ford-f-150/?sponsoredVehiclePosition=2&zipcode=07606',
 'https://www.truecar.com/used-cars-for-sale/listing/1G1ZD5ST7LF088152/2020-chevrolet-malibu/?zipcode=33619',
 'https://www.truecar.com/used-cars-for-sale/listing/1FTFW1E8XMKD49013/2021-ford-f-150/?zipcode=33584',
 'https://www.truecar.com/used-cars-for-sale/listing/1FTEW1EB9MKE00573/2021-ford-f-150/?zipcode=42728',
 'https://www.truecar.com/used-cars-for-sale/listing/1FTEW1EP2MKD38719/2021-ford-f-150/?zipcode=61081',
 'https://www.truecar.com/used-cars-for-sale/listing/1FTEW1E57LFA96953/2020-ford-f-150/?zipcode=14625',
 'https://www.truecar.com/used-cars-for-sale/listing/5YFEPMAE8MP180813

## Returning the raw dataframe

In [34]:
%%time
data = page_scraping(urls[:5])

Year=2017, Make=Mercedes-Benz, Model=C-Class, Sub model:C 300 Sedan RWD, City=Riverside, State=CA, Mileage=62,840, Price=16,837    , Exterior_color=Selenite Gray Metallic
, interior_color=Ash, mpg_city=23 mpg_hwy=30, Engine=2.0L Inline-4 Gas Inline,     Transmission=Automatic Transmission, Drive_type=RWD, Fuel_type=Gas,
 Popular_feature=, accidents=1, Owners=2
Year=2020, Make=Ford, Model=F-150, Sub model:XLT SuperCrew 5.5' Box 4WD, City=Michigan Center, State=MI, Mileage=49,089, Price=31,601    , Exterior_color=White
, interior_color=Medium Light Camel, mpg_city=18 mpg_hwy=23, Engine=5.0L V-8 Gas V,     Transmission=Automatic Transmission, Drive_type=4WD, Fuel_type=Gas,
 Popular_feature=, accidents=0, Owners=0
Year=2016, Make=Ford, Model=F-150, Sub model:XLT SuperCrew 5.5' Box 4WD, City=Garfield, State=NJ, Mileage=179,565, Price=13,199    , Exterior_color=Black
, interior_color=Black, mpg_city=17 mpg_hwy=23, Engine=2.7L V-6 Gas V,     Transmission=Automatic Transmission, Drive_type=4WD

In [35]:
print(data.shape)
data.head(20)

(5, 19)


,year,make,model,sub_model,city,state,mileage,price,exterior_color,interior_color,mpg_city,mpg_hwy,engine,transmission,drive_type,fuel_type,popular_feature,accidents,owners
0,2017,Mercedes-Benz,C-Class,C 300 Sedan RWD,Riverside,CA,"62,840","16,837",Selenite Gray Metallic,Ash,23,30,2.0L Inline-4 Gas Inline,Automatic Transmission,RWD,Gas,,1,2
1,2020,Ford,F-150,XLT SuperCrew 5.5' Box 4WD,Michigan Center,MI,"49,089","31,601",White,Medium Light Camel,18,23,5.0L V-8 Gas V,Automatic Transmission,4WD,Gas,,0,0
2,2016,Ford,F-150,XLT SuperCrew 5.5' Box 4WD,Garfield,NJ,"179,565","13,199",Black,Black,17,23,2.7L V-6 Gas V,Automatic Transmission,4WD,Gas,,0,1
3,2020,Chevrolet,Malibu,LT,Tampa,FL,"64,185","14,066",Mosaic Black Metallic,Jet Black,29,36,1.5L Inline-4 Gas Inline,Automatic Transmission,FWD,Gas,", Premium Wheels, Apple CarPlay, Android Auto,...",2,2
4,2021,Ford,F-150,XLT SuperCrew 5.5' Box 4WD,Seffner,FL,"31,360","30,484",Oxford White,Black/Medium Dark Slate,19,22,3.5L V-6 Gas V,Automatic Transmission,4WD,Gas,,0,1


In [ ]:
data.to_csv('../data/usedCarListing.csv', encoding = 'utf-8')